In [7]:
import os
import cv2
import torch
import numpy as np
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

def load_model(config_file, weights_file):
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.95
    return DefaultPredictor(cfg)

def find_outer_contour(mask):
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def find_convex_hull(contour):
    return cv2.convexHull(contour)

def fit_lines_from_convex_hull(hull, num_lines):
    epsilon = 0.02 * cv2.arcLength(hull, True)
    approx = cv2.approxPolyDP(hull, epsilon, closed=True)
    
    lines = []
    if len(approx) >= 2:
        for i in range(len(approx) - 1):
            lines.append([approx[i][0], approx[i+1][0]])
        lines.append([approx[-1][0], approx[0][0]])
    
    return lines[:num_lines]

def calculate_angle(x1, y1, x2, y2):
    return np.degrees(np.arctan2(y2 - y1, x2 - x1))

def extend_lines(lines, extension_length, min_angle=0, max_angle=180):
    largest_angle = -1
    largest_angle_line = None
    for line in lines:
        x1, y1 = line[0]
        x2, y2 = line[1]
        
        dx = x2 - x1
        dy = y2 - y1
        
        angle = calculate_angle(x1, y1, x2, y2)
        
        if min_angle <= angle <= max_angle and angle > largest_angle:
            largest_angle = angle
            largest_angle_line = [(int(x1 - extension_length * dx), int(y1 - extension_length * dy)),
                                  (int(x2 + extension_length * dx), int(y2 + extension_length * dy))]
    
    return [(largest_angle_line, largest_angle)] if largest_angle_line else []

def draw_lines(image, lines):
    for line, angle in lines:
        cv2.line(image, tuple(line[0]), tuple(line[1]), (0, 255, 0), 2)
        mid_x = (line[0][0] + line[1][0]) // 2
        mid_y = (line[0][1] + line[1][1]) // 2
        cv2.putText(image, f"{angle:.1f}", (mid_x, mid_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
    return image

def visualize_masks_with_extended_lines(image, instances, num_lines=5, extension_length=20, min_angle=0, max_angle=180):
    masks = instances.pred_masks.cpu().numpy()
    lines = []
    
    for mask in masks:
        contours = find_outer_contour(mask)
        for contour in contours:
            hull = find_convex_hull(contour)
            lines.extend(fit_lines_from_convex_hull(hull, num_lines))
    
    extended_lines = extend_lines(lines, extension_length, min_angle, max_angle)
    result_image = draw_lines(image.copy(), extended_lines)
    
    return result_image

def draw_and_segment_quadrilaterals(image, instances, deformation_threshold=0.05):
    masks = instances.pred_masks.cpu().numpy()
    result_image = image.copy()
    
    # Create a blank mask for all class 1 objects
    all_class_1_mask = np.zeros(image.shape[:2], dtype=np.uint8)
    
    for i, mask in enumerate(masks):
        contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contours) == 0:
            continue
        
        for contour in contours:
            epsilon = deformation_threshold * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            if len(approx) == 4:
                # Draw the quadrilateral
                cv2.polylines(result_image, [approx], True, (0, 255, 0), 2)
                
                # Add this object's mask to the all_class_1_mask
                all_class_1_mask = cv2.bitwise_or(all_class_1_mask, mask.astype(np.uint8))
                
                # Label the quadrilateral
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    cv2.putText(result_image, f"Tank {i+1}", (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    # Apply a semi-transparent highlight to all class 1 objects
    highlight_color = (0, 0, 255)  # Red color for highlighting
    highlight_alpha = 0.3  # Transparency of the highlight
    
    highlight_mask = np.zeros(image.shape, dtype=np.uint8)
    highlight_mask[all_class_1_mask == 1] = highlight_color
    
    cv2.addWeighted(highlight_mask, highlight_alpha, result_image, 1 - highlight_alpha, 0, result_image)
    
    return result_image

def process_image(predictor, image_path, min_angle=0, max_angle=180):
    image = cv2.imread(image_path)
    outputs = predictor(image)
    
    instances = outputs["instances"].to("cpu")
    classes = instances.pred_classes.numpy()
    
    result_image = image.copy()
    
    # Process class 1 (quadrilaterals)
    class_1_instances = instances[classes == 1]
    if len(class_1_instances) > 0:
        result_image = draw_and_segment_quadrilaterals(result_image, class_1_instances)
    
    # Process class 2 (extended lines)
    class_2_instances = instances[classes == 2]
    if len(class_2_instances) > 0:
        result_image = visualize_masks_with_extended_lines(result_image, class_2_instances, min_angle=min_angle, max_angle=max_angle)
    
    cv2.imshow(f'Result for {image_path}', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(predictor, video_path, min_angle=0, max_angle=180, target_fps=5
                  
                  ):
    cap = cv2.VideoCapture(video_path)
    original_fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_interval = max(1, round(original_fps / target_fps))
    
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        if frame_count % frame_interval != 0:
            continue
        
        outputs = predictor(frame)
        instances = outputs["instances"].to("cpu")
        classes = instances.pred_classes.numpy()
        
        result_frame = frame.copy()
        
        # Process class 1 (quadrilaterals)
        class_1_instances = instances[classes == 1]
        if len(class_1_instances) > 0:
            result_frame = draw_and_segment_quadrilaterals(result_frame, class_1_instances)
        
        # Process class 2 (extended lines)
        class_2_instances = instances[classes == 2]
        if len(class_2_instances) > 0:
            result_frame = visualize_masks_with_extended_lines(result_frame, class_2_instances, min_angle=min_angle, max_angle=max_angle)
        
        cv2.imshow(f'Result for a frame in {video_path}', result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

def process_folder(predictor, folder_path, min_angle=0, max_angle=180, target_fps=10):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                process_image(predictor, file_path, min_angle=min_angle, max_angle=max_angle)
            elif file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                process_video(predictor, file_path, min_angle=min_angle, max_angle=max_angle, target_fps=target_fps)


# Example usage
config_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_and_edge/tank-and-ramp-1/mask_rcnn_R_101_FPN_3x/2024-07-02-09-05-45/config.yaml"
weights_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_and_edge/tank-and-ramp-1/mask_rcnn_R_101_FPN_3x/2024-07-02-09-05-45/model_final.pth"
input_folder = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/test"


predictor = load_model(config_file, weights_file)

min_angle = 40
max_angle = 135
process_folder(predictor, input_folder, min_angle, max_angle)

In [9]:
import os
import cv2
import torch
import numpy as np
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

def load_model(config_file, weights_file):
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.WEIGHTS = weights_file
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.95
    return DefaultPredictor(cfg)

def find_outer_contour(mask):
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def find_convex_hull(contour):
    return cv2.convexHull(contour)

def fit_lines_from_convex_hull(hull, num_lines):
    epsilon = 0.02 * cv2.arcLength(hull, True)
    approx = cv2.approxPolyDP(hull, epsilon, closed=True)
    
    lines = []
    if len(approx) >= 2:
        for i in range(len(approx) - 1):
            lines.append([approx[i][0], approx[i+1][0]])
        lines.append([approx[-1][0], approx[0][0]])
    
    return lines[:num_lines]

def calculate_angle(x1, y1, x2, y2):
    return np.degrees(np.arctan2(y2 - y1, x2 - x1))

def extend_lines(lines, extension_length, min_angle=0, max_angle=180):
    largest_angle = -1
    largest_angle_line = None
    for line in lines:
        x1, y1 = line[0]
        x2, y2 = line[1]
        
        dx = x2 - x1
        dy = y2 - y1
        
        angle = calculate_angle(x1, y1, x2, y2)
        
        if min_angle <= angle <= max_angle and angle > largest_angle:
            largest_angle = angle
            largest_angle_line = [(int(x1 - extension_length * dx), int(y1 - extension_length * dy)),
                                  (int(x2 + extension_length * dx), int(y2 + extension_length * dy))]
    
    return [(largest_angle_line, largest_angle)] if largest_angle_line else []

def draw_lines(image, lines):
    for line, angle in lines:
        cv2.line(image, tuple(line[0]), tuple(line[1]), (0, 255, 0), 2)
        mid_x = (line[0][0] + line[1][0]) // 2
        mid_y = (line[0][1] + line[1][1]) // 2
        cv2.putText(image, f"{angle:.1f}", (mid_x, mid_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
    return image

def visualize_masks_with_extended_lines(image, instances, num_lines=5, extension_length=20, min_angle=0, max_angle=180):
    masks = instances.pred_masks.cpu().numpy()
    lines = []
    
    for mask in masks:
        contours = find_outer_contour(mask)
        for contour in contours:
            hull = find_convex_hull(contour)
            lines.extend(fit_lines_from_convex_hull(hull, num_lines))
    
    extended_lines = extend_lines(lines, extension_length, min_angle, max_angle)
    result_image = draw_lines(image.copy(), extended_lines)
    
    return result_image

def draw_and_segment_quadrilaterals(image, instances, deformation_threshold=0.05):
    masks = instances.pred_masks.cpu().numpy()
    result_image = image.copy()
    
    # Create a blank mask for all class 1 objects
    all_class_1_mask = np.zeros(image.shape[:2], dtype=np.uint8)
    
    for i, mask in enumerate(masks):
        contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contours) == 0:
            continue
        
        for contour in contours:
            epsilon = deformation_threshold * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            if len(approx) == 4:
                # Draw the quadrilateral
                cv2.polylines(result_image, [approx], True, (0, 255, 0), 2)
                
                # Add this object's mask to the all_class_1_mask
                all_class_1_mask = cv2.bitwise_or(all_class_1_mask, mask.astype(np.uint8))
                
                # Label the quadrilateral
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    cv2.putText(result_image, f"Tank {i+1}", (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    # Apply a semi-transparent highlight to all class 1 objects
    highlight_color = (0, 0, 255)  # Red color for highlighting
    highlight_alpha = 0.3  # Transparency of the highlight
    
    highlight_mask = np.zeros(image.shape, dtype=np.uint8)
    highlight_mask[all_class_1_mask == 1] = highlight_color
    
    cv2.addWeighted(highlight_mask, highlight_alpha, result_image, 1 - highlight_alpha, 0, result_image)
    
    return result_image

def process_image(predictor, image_path, output_path, min_angle=0, max_angle=180):
    image = cv2.imread(image_path)
    outputs = predictor(image)
    
    instances = outputs["instances"].to("cpu")
    classes = instances.pred_classes.numpy()
    
    result_image = image.copy()
    
    # Process class 1 (quadrilaterals)
    class_1_instances = instances[classes == 1]
    if len(class_1_instances) > 0:
        result_image = draw_and_segment_quadrilaterals(result_image, class_1_instances)
    
    # Process class 2 (extended lines)
    class_2_instances = instances[classes == 2]
    if len(class_2_instances) > 0:
        result_image = visualize_masks_with_extended_lines(result_image, class_2_instances, min_angle=min_angle, max_angle=max_angle)
    
    cv2.imwrite(output_path, result_image)

def process_video(predictor, video_path, output_folder, min_angle=0, max_angle=180, target_fps=5):
    cap = cv2.VideoCapture(video_path)
    original_fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_interval = max(1, round(original_fps / target_fps))
    
    frame_count = 0
    output_video_path = os.path.join(output_folder, os.path.basename(video_path))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if out is None:
            frame_height, frame_width = frame.shape[:2]
            out = cv2.VideoWriter(output_video_path, fourcc, target_fps, (frame_width, frame_height))
        
        frame_count += 1
        if frame_count % frame_interval != 0:
            continue
        
        outputs = predictor(frame)
        instances = outputs["instances"].to("cpu")
        classes = instances.pred_classes.numpy()
        
        result_frame = frame.copy()
        
        # Process class 1 (quadrilaterals)
        class_1_instances = instances[classes == 1]
        if len(class_1_instances) > 0:
            result_frame = draw_and_segment_quadrilaterals(result_frame, class_1_instances)
        
        # Process class 2 (extended lines)
        class_2_instances = instances[classes == 2]
        if len(class_2_instances) > 0:
            result_frame = visualize_masks_with_extended_lines(result_frame, class_2_instances, min_angle=min_angle, max_angle=max_angle)
        
        out.write(result_frame)
    
    cap.release()
    out.release()

def process_folder(predictor, folder_path, output_folder, min_angle=0, max_angle=180, target_fps=10):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            output_path = os.path.join(output_folder, file_name)
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                process_image(predictor, file_path, output_path, min_angle=min_angle, max_angle=max_angle)
            elif file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                process_video(predictor, file_path, output_folder, min_angle=min_angle, max_angle=max_angle, target_fps=target_fps)

# Example usage
config_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_and_edge/tank-and-ramp-1/mask_rcnn_R_101_FPN_3x/2024-07-02-09-05-45/config.yaml"
weights_file = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/ramp_and_edge/tank-and-ramp-1/mask_rcnn_R_101_FPN_3x/2024-07-02-09-05-45/model_final.pth"
input_folder = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/test"
output_folder = "C:/Users/Daanish Mittal/OneDrive/Desktop/Tank_align/tank_alignment/output"

predictor = load_model(config_file, weights_file)

min_angle = 40
max_angle = 135
process_folder(predictor, input_folder, output_folder, min_angle, max_angle)
